In [ ]:
"""Not used in paper. Used the hyperparameters trained in DeiT paper instead"""
import sys
import os
sys.path.append('../')

In [ ]:
from TrainFunctions.train_resnet import train_resnet
from functools import partial
from ray import tune
import ray
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter

In [ ]:
ray.shutdown()
os.environ['PYTHONPATH'] = '/Users/adithyagiri/Desktop/ConvFormer/Repo/'
ray.init(num_gpus=18, num_cpus=12)
config = {
        "lr": tune.choice([0.003, 0.01, 0.03, 0.1, 0.3]),
        "weight_decay": tune.choice([0.001, 0.003, 0.01, 0.03]),
        "momentum": tune.choice([0.9, 0.95, 0.99]),
        'lr_scheduler': tune.choice(['OneCycleLR']),
    }
max_epochs = 32
scheduler = ASHAScheduler(
        metric="accuracy",
        mode="max",
        max_t=max_epochs,
        reduction_factor=2,
        grace_period=max_epochs/4,
        brackets = 1,
)
reporter = CLIReporter(
        metric_columns=["loss", "accuracy", "training_iteration"])
result = tune.run(
    partial(train_resnet, n = 18, tuning_mode = True),
    config = config,
    num_samples = 16,
    scheduler = scheduler,
    progress_reporter = reporter,
    max_concurrent_trials=2
)




In [ ]:
best_config = result.get_best_config(metric="accuracy", mode="max")
print("Best Hyperparameters:", best_config)
